Random DNA sample from https://www.bioinformatics.org/sms2/random_dna.html
Following tutorial on https://www.kaggle.com/thomasnelson/working-with-dna-sequence-data-for-ml/data
Bio Python help from https://biopython.org/wiki/SeqRecord

Qs
One-hot vs. ordinal encoding -> no diff?
Why are the genomes different lengths?
# Find median length and exclude rest ? Trim sequences?
Feature extraction? https://www.quora.com/What-is-the-common-practice-to-handle-different-lengths-of-DNA-sequence-datasets-in-machine-learning-classification-task

In [ ]:
!pip3 install pandas
!pip3 install biopython

In [ ]:
# All imports
import pandas as pd
from Bio import SeqIO # FASTA reader


In [ ]:
for index, seq_record in enumerate(SeqIO.parse('./test_dna.fasta', "fasta")):
    print("index %i, ID = %s, length %i, with %i features"
          % (index, seq_record.id, len(seq_record.seq), len(seq_record.features)))

In [ ]:
for index, seq_record in enumerate(SeqIO.parse('/Users/alevenberg/Documents/research/VirLab/genomes/Train_Culex/Japanese_encephalitis_virus.fasta', "fasta")):
    print("index %i, ID = %s, length %i, with %i features"
          % (index, seq_record.id, len(seq_record.seq), len(seq_record.features)))
   # print(seq_record)
   # print(seq_record.format("fasta"))
    


In [ ]:
for index, seq_record in enumerate(SeqIO.parse('/Users/alevenberg/Documents/research/VirLab/genomes/Train_Culex/west_nile_virus.fasta', "fasta")):
    print("index %i, ID = %s, length %i, with %i features"
          % (index, seq_record.id, len(seq_record.seq), len(seq_record.features)))
   # print(seq_record)

In [ ]:
# function to convert a DNA sequence string to a numpy array
# converts to lower case, changes any non 'acgt' characters to 'n'
import numpy as np
import re
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

# create a label encoder with 'acgtn' alphabet
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

In [ ]:
# function to one-hot encode a DNA sequence string
# non 'acgt' bases (n) are 0000
# returns a L x 4 numpy array
from sklearn.preprocessing import OneHotEncoder
def one_hot_encoder(my_array):
    integer_encoded = label_encoder.transform(my_array)
   # print(integer_encoded) # A -> 0, C -> 1, G -> 2, T -> 3 and N -> 4
    onehot_encoder = OneHotEncoder(sparse=False, dtype=int,categories=[range(5)])
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1) # Makes the one array -> 2D array of single elements
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded) # 0 -> 1,0,0,0,0, 1 -> 0,1,0,0,0
   # print(onehot_encoded)
    onehot_encoded = np.delete(onehot_encoded, -1, 1)
    #print(onehot_encoded) # Deletes last column so those become 0,0,0,0 if there was a non ACGT character
    return onehot_encoded

test_sequence = 'AACGCGGTTNN'
one_hot_encoder(string_to_array(test_sequence))

In [ ]:
import pandas as pd


mySeq = 'CATGGCCATCCCCCCCCGAGCGGGGGGGGGG'
getKmers(mySeq, size=2)
words = getKmers(mySeq, size=2)
sentence = ' '.join(words)


mySeq2 = 'GATGGCCATCCCCGCCCGAGCGGGGGGGG'
mySeq3 = 'CATGGCCATCCCCGCCCGAGCGGGCGGGG'
sentence2 = ' '.join(getKmers(mySeq2, size=2))
sentence3 = ' '.join(getKmers(mySeq3, size=2))

sentence3

# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform([sentence, sentence2, sentence3]).toarray() # holds the kmers and their counts
print(sentence)
print(X)
print(type(X))
tokens = cv.get_feature_names()
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=X, index=['Seq', 'Seq2', 'Seq3'], columns=tokens).head()

In [ ]:
for index, seq_record in enumerate(SeqIO.parse('/Users/alevenberg/Documents/research/VirLab/genomes/Train_Culex/west_nile_virus.fasta', "fasta")):
    print("index %i, ID = %s"
          % (index, seq_record.id))
    print(one_hot_encoder(string_to_array(str(seq_record.seq))))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


sequences = []
ids = []

for index, seq_record in enumerate(SeqIO.parse('/Users/alevenberg/Documents/research/VirLab/genomes/Train_Culex/west_nile_virus.fasta', "fasta")):
    sequences.append(getKmersStr(str(seq_record.seq), size=6))
    ids.append(seq_record.id)
    
cv = CountVectorizer()
X = cv.fit_transform(sequences).toarray() # holds the kmers and their counts
tokens = cv.get_feature_names()
pd.DataFrame(data=X, index=ids, columns=tokens).head()

Current work below

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from Bio import SeqIO # FASTA reader

paths = ["/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Dengue_virus.fasta", "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Yellow_fever_virus.fasta", "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Zika_virus.fasta"]

KMERS =3

def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

def getKmersStr(sequence, size):
    kmers = getKmers(sequence, size)
    return " ".join(kmers)

# Aedes is a vector
# Each vectors dna sample has the same # of kmers??
cv = CountVectorizer()

for path in paths:
    sequences = []
    ids = []
    for index, seq_record in enumerate(SeqIO.parse(path, "fasta")):
        sequences.append(getKmersStr(str(seq_record.seq), size=KMERS))
        ids.append(seq_record.id)
    
    X = cv.fit_transform(sequences).toarray() # holds the kmers and their counts
    tokens = cv.get_feature_names()
    df = pd.DataFrame(data=X, index=ids, columns=tokens)
    print(df.loc[:, (df != 0).any(axis=0)])
    print("Finished path {}".format(path))

prints out kmers 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from Bio import SeqIO # FASTA reader

path = "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Dengue_virus.fasta"

KMERS = 3

def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

def getKmersStr(sequence, size):
    kmers = getKmers(sequence, size)
    return " ".join(kmers)

# Aedes is a vector
# Each vectors dna sample has the same # of kmers??
cv = CountVectorizer()

sequences = []
ids = []
for index, seq_record in enumerate(SeqIO.parse(path, "fasta")):
    if ("aab" in str(seq_record.seq)):
        print(seq_record.id)
    sequences.append(getKmersStr(str(seq_record.seq), size=KMERS))
    ids.append(seq_record.id)

X = cv.fit_transform(sequences).toarray() # holds the kmers and their counts
tokens = cv.get_feature_names()
df = pd.DataFrame(data=X, index=ids, columns=tokens)
df = df.loc[:, (df != 0).any(axis=0)]


print("Finished path {}".format(path))

In [ ]:
df = df.loc[:, (df != 0).any(axis=0)]
print(df)


df = df.loc[:, df.isin([' ','NULL',0]).mean() < 1]
print (df)
# iterating the columns 
for col in df.columns: 
    print(col)

In [ ]:
def toString(List): 
    return ''.join(List) 
  
# Function to print permutations of string 
# This function takes three parameters: 
# 1. String 
# 2. Starting index of the string 
# 3. Ending index of the string. 
def permute(a, l, r): 
    if l==r: 
        print (toString(a) )
    else: 
        for i in range(l,r+1): 
            a[l], a[i] = a[i], a[l] 
            permute(a, l+1, r) 
            a[l], a[i] = a[i], a[l] # backtrack 
  
# Driver program to test the above function 
string = "acgt"
n = len(string) 
a = list(string) 
permute(a, 0, n-1) 
  

In [ ]:
# A Python program to print all  
# combinations of given length 
from itertools import combinations 
from itertools import permutations 
from itertools import product

# Get all combinations of [1, 2, 3] 
# and length 2 
comb = product(['a','c','g','t'],repeat=3) 
  
cart = []
# Print the obtained combinations 
for i in list(comb): 
    cart.append("".join(i))
    print("".join(i)) 

In [ ]:
cols = []
for col in df.columns: 
    print(col)
    cols.append(col)

In [ ]:
cols == cart # all possible combinations are shown which is good
# if there was a kmer which was not found it might have accidentally been dropped earlier!

In [ ]:
print (df)

In [ ]:
df.value_counts().sort_index().plot.bar()


***************

In [39]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from Bio import SeqIO # FASTA reader
import numpy as np

path = "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Aedes/Dengue_virus.fasta"

# Get Sequences
sequences = []
ids = []
for index, seq_record in enumerate(SeqIO.parse(path, "fasta")):
    sequences.append(str(seq_record.seq))
    ids.append(seq_record.id)                 

In [42]:
sequences
classes = np.full(shape=len(sequences),fill_value=0)
print(len(classes))
print(len(sequences))

5446
5446


In [37]:
a = np.array([1,2,3,4,5])
b = np.array([6,7,8,9,10])
np.array(list(zip(a,b)))


array([[ 1,  6],
       [ 2,  7],
       [ 3,  8],
       [ 4,  9],
       [ 5, 10]])

In [41]:
numpy_data = np.array(list(zip(sequences,classes)))
df_aedes = pd.DataFrame(data=numpy_data)
df_aedes.columns = ["sequence","class"]
#({"columns":})
df_aedes.head()

,sequence,class
0,CTAACAGTTTTTTATTAGAGAGCAGATCTCTGATGAACAACCAACG...,0
1,CTAACAGTTTTTTATTAGAGAGCAGATCTCTGATGAACAACCAACG...,0
2,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
3,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGACTCGGAAG...,0
4,ATGAATAACCAACGAAAAAAGGCGAGAAATACGCCTTTCAATATGC...,0


In [45]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from Bio import SeqIO # FASTA reader
import numpy as np

path = "/Users/alevenberg/Documents/research/VirLab/genomes/Full_Culex/Japanese_encephalitis_virus.fasta"

# Get Sequences
sequences = []
ids = []
for index, seq_record in enumerate(SeqIO.parse(path, "fasta")):
    sequences.append(str(seq_record.seq))
    ids.append(seq_record.id)                 

In [46]:
classes = np.full(shape=len(sequences),fill_value=1)
print(len(classes))
print(len(sequences))

324
324


In [47]:
numpy_data = np.array(list(zip(sequences,classes)))
df_culex = pd.DataFrame(data=numpy_data)
df_culex.columns = ["sequence","class"]
#({"columns":})
df_culex.head()

,sequence,class
0,AGAAGTTTATCTGTGTGGACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
1,AGAAGTTTATCTGTGTGGACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
2,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
3,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1
4,AGAAGTTTATCTGTGTGAACTTCTTGGCTTAGTATCGTTGAGAAGA...,1


In [57]:
frames = [df_aedes, df_culex]

full_df = df_aedes.append(df_culex)


In [58]:
full_df.tail()
full_df.shape

(5770, 2)

In [59]:
df_aedes.tail()


,sequence,class
5441,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
5442,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
5443,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
5444,AGTTGTTAGTCTACGTGGACCGACAAGAACAGTTTCGAATCGGAAG...,0
5445,GTGGACCGCAAAGAACAGTTTCGAATCGGAAGCTTGCTTAACGTAG...,0


In [1]:
df_aedes.shape
df_aedes.new_shape(324,2)
df_aedes.shape

NameError: name 'df_aedes' is not defined

In [60]:
df_culex.shape

(324, 2)

In [ ]:
                 

KMERS = 3

def getKmers(sequence, size):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

def getKmersStr(sequence, size):
    kmers = getKmers(sequence, size)
    return " ".join(kmers)

# Aedes is a vector
# Each vectors dna sample has the same # of kmers??
cv = CountVectorizer()



X = cv.fit_transform(sequences).toarray() # holds the kmers and their counts
tokens = cv.get_feature_names()
df = pd.DataFrame(data=X, index=ids, columns=tokens)
df = df.loc[:, (df != 0).any(axis=0)]


print("Finished path {}".format(path))